In [ ]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from utils import flatten, buffered
import re
from difflib import unified_diff

class File:
    def __init__(self, path, buffer = []):
        self.path = path
        self.buffer = buffer

    @classmethod
    def from_editList(cls, path, editList):
        buffer = [
            txt.replace('\n', '')
            for edit in editList
                for doc in edit if (txt:=doc.get("text", None)) is not None
        ]
        return cls(
            path = path,
            buffer = buffer
        )
    
    def insert(self, start, end, text:str):
        buffer = self.buffer
        (startLineNumber, startColumn) = start
        (endLineNumber, endColumn) = end
        lines = text.splitlines()
        if (nLines:=endLineNumber - len(buffer)) > 0:
            buffer.extend([""]*nLines)
        for i, s in zip(
            range(startLineNumber, endLineNumber + 1),
            lines
        ):
            if i == startLineNumber: 
                line = buffer[i-1]
                buffer[i-1] = line[:startColumn] + s
            elif i == endLineNumber:
                line = buffer[i-1]
                buffer[i-1] = s + line[endColumn:]
            else: pass
                



class Chat:    
    files = defaultdict(list)
    @classmethod
    def extract_attachments(cls, metadata_lst):
        for text in (
            txt
            for meta in metadata_lst if (msgs:=meta.get("renderedUserMessage", []))
                for msg in msgs if (txt:=msg.get("text", ""))
        ):
            # wrap codeblocks to escape < and > in code
            text = re.sub(r"\n```\S*\n(?P<code>[^`]*?)```", r"<![CDATA[\g<code>]]>", text)
            text = "<root>" + text + "</root>"
            try: root = ET.fromstring(text)
            except ET.ParseError as e:
                print("Error parsing attachments:")
                print(e)
                continue

            for attachment in root.findall(".//attachment"):
                filePath = attachment.attrib.get("filePath", None)
                if filePath is not None:
                    cls.files[filePath].insert(
                        0, 
                        File(
                            path = filePath,
                            buffer = attachment.text.splitlines()
                        )
                    )
                else:
                    print(f"Encountered unknown attachment\n{attachment}")


    def __init__(self, doc):
        requesterUsername = doc["requesterUsername"]
        responderUsername = doc["responderUsername"]
        metadata_lst = (
            metadata
            for request in doc.get("requests", [])
            if (
                metadata:=request
                .get("result", {})
                .get("metadata", {})
            )
        )
        self.extract_attachments(metadata_lst)
        
        self.requests = [
            Request(
                req,
                requesterUsername=requesterUsername,
                responderUsername=responderUsername
                ) 
            for req in doc["requests"]
            ]
        
        
    def render(self):
        return map(
            lambda line: line + "\n",
            flatten(
                map(
                    lambda req: req.render(),
                    self.requests
                )
            )
        )
     
class VariableData:
    def __init__(self, doc):
        self.variables = []
        for var in doc["variables"]:
            kind = doc.get("kind", None)
            if kind == "file":
                self.variables.append(f"file: `{doc['name']}`")
            else:
                print(f"Unknown variable encountered:\n{doc}")
    def render(self):
        return ", ".join(self.variables)

class Request:
    from arango import ArangoClient
    coll = ArangoClient("http://localhost:8529").db().collection("chat-ids")
    @classmethod
    def get_model(cls, responseId):
        cursor = cls.coll.find({"request_id": responseId}, skip=0, limit=1)
        if cursor.empty(): return None
        else: return cursor.next()["model"]
    
    def __init__(self, doc, *, requesterUsername, responderUsername):
        self.requesterUsername = requesterUsername
        self.responderUsername = responderUsername
        self.responseId = doc["result"]["metadata"]["responseId"]
        self.model = self.get_model(self.responseId)
        self.modes = doc["agent"]["modes"]
        self.message = doc["message"]["text"]
        self.response = Response(doc["response"])
        self.variableData = VariableData(doc["variableData"])

    def render(self):
        return [
            map(
                lambda s: s + "\n",
                [
                    f"> {self.requesterUsername}:",
                    self.message,
                    *([s] if (s:=self.variableData.render()) else []),
                    "",
                    "> " + self.responderUsername + (f" ({self.model})" if self.model else "") + ':'   
                ]
            ),
            self.response.render()
        ]
    
class Response:
    def __init__(self, lst):
        self.chunks = []
        it = buffered(lst)
        for chunk in it:
            obj = None
            
            kind = chunk.get("kind", None)
            #tool invocation
            if kind=="toolInvocationSerialized":
                if chunk["toolId"] == "copilot_insertEdit":
                    it.enqueue(chunk)
                    obj = CopilotFileEdit(it)
                
            #text block
            elif (
                ("value" in chunk and kind is None) or
                kind == "inlineReference"
            ):
                it.enqueue(chunk)
                obj = TextBlock(it)
            
            if obj:
                self.chunks.append(obj)
            else:
                print(f"Unknown chunk encountered:\n{chunk}")
    
    def render(self):
        return map(lambda chunk: chunk.render(), self.chunks)

class TextBlock:
    def __init__(self, it):
        self.chunks = []
        for chunk in it:
            kind = chunk.get("kind", None)
            if kind == "inlineReference":
                self.chunks.append(InlineReference(chunk))
            elif "value" in chunk and kind is None:
                self.chunks.append(TextChunk(chunk))
            else:
                it.enqueue(chunk)
                break

    def render(self):
        return (
            ''.join( 
                map(
                    lambda chunk: chunk.render(),
                    self.chunks
                ))
            ).splitlines()
        
class TextChunk:
    def __init__(self, doc):
        self.text = doc["value"]
    def render(self):
        return self.text

class CopilotFileEdit:
    def __init__(self, it):
        self.chunks = []
        for obj in it:
            self.chunks.append(obj)
            if (
                obj.get("kind", None) == "textEditGroup"
                and obj["done"] == True
            ): break

        
        editGroups = filter(
            lambda chunk: chunk.get("kind", None) == "textEditGroup",
            self.chunks
        )
        
        self.fileEdits = []

        for fileEdit in editGroups:
            path = fileEdit["uri"]["path"]
            lst = Chat.files[path]
            lst.append(File.from_editList(
                path = path,
                editList= fileEdit["edits"]
            ))
            self.fileEdits.append((path, len(lst)-1))        

    def render(self):
        for (path, i) in self.fileEdits:
            yield f"\nEdited file `{path}`:"
            if i > 0:
                lst = Chat.files[path]
                yield from [
                    "```diff",
                    unified_diff(lst[i-1].buffer, lst[i].buffer, lineterm=""),
                    "```"
                ]

class InlineReference:
    def __init__(self, doc):
        ref = doc["inlineReference"]
        self.text = ""
        #file reference
        if "path" in ref:
            self.text = "file `" + ref["path"] + '`'
        #symbol reference
        elif "name" in ref:
            self.text = '`' + ref["name"] + '`'
        
    def render(self):
        return self.text

In [ ]:
doc = {}
with open("/home/p/Philip.Obi/chat-renderer/chat.json", "r") as f:
    import json
    doc = json.load(f)

In [ ]:
chat = Chat(doc)

In [ ]:
with open("chat.md", "w") as f:
    f.writelines(chat.render())

In [ ]:
Chat.files